# Cluster Genotypes for Training set Definition

> 

<span style="color:red"> *NOTE:*</span> This notebook includes visualization of clusters produced with HDBSCAN. At present I'm not using this approach but see [rapid's cuML's documentation](https://docs.rapids.ai/api/cuml/stable/api/#hdbscan) for more information about this method. 

In [ ]:
import os
import re

import numpy as np
import pandas as pd

import plotly.express as px

from tqdm import tqdm

import json
import time

from dataG2F.qol import ensure_dir_path_exists

In [ ]:
cache_path = '../nbs_artifacts/08_cluster_genotypes/'
ensure_dir_path_exists(dir_path = cache_path)

## Load phenotypic data to match

In [ ]:
load_from = '../nbs_artifacts/05_prep_matrices/'
phno = pd.read_csv(load_from+'phno_geno.csv')
phno

## Clusters based on HDBSCAN of genomes.

In [ ]:
# if the hdbscan doesn't exist, make it.
if not (os.path.exists(cache_path+'hdbscan_acgt_clustering_min_samples.npy') and 
        os.path.exists(cache_path+'hdbscan_acgt_clustering.npy')):
    from cuml.cluster import HDBSCAN # this is in the rapids enviroment
    load_from = '../nbs_artifacts/01.03_g2fc_prep_matrices/'
    ACGT = np.load(load_from+'ACGT.npy')

    n_obs = ACGT.shape[0]
    n_len = np.prod(ACGT.shape[1:3])
    ACGT = ACGT.swapaxes(1,2  # swap so that when the the array is reshaped it's in the order 1ACGT 2ACGT ... instead of 1A2A3A ...
                 ).reshape(n_obs, n_len)    
    
    min_samples = [(2+i) for i in range(23)]+[25*(1+i) for i in range(10)]

    # This may need to be run a few times. Memory on the gpu isn't being freed as soon as it's no longer needed. 
    # Alternately an arbitrary delay could be used.
    for i in tqdm(range(len(min_samples))):
        # print(min_samples[i])
        save_res_path = cache_path+'hdbscan_acgt_num_'+str(min_samples[i])+'.npy'
        if not os.path.exists(save_res_path):
            labels_ = HDBSCAN(min_samples=min_samples[i]).fit_predict(ACGT)
            np.save(save_res_path, labels_)

    temp =  [e for e in os.listdir(cache_path) if re.match('hdbscan_acgt_num_.+', e)]
    temp_names = [e.replace('hdbscan_acgt_num_', '').replace('.npy', '') for e in temp]

    temp = np.concatenate([np.load(cache_path+e)[:, None] for e in temp], axis = 1)

    np.save(cache_path+'hdbscan_acgt_clustering_min_samples.npy', np.array(min_samples))
    np.save(cache_path+'hdbscan_acgt_clustering.npy', np.array(temp))

else:    
    hdbscan_acgt_clustering_min_samples = np.load(cache_path+'hdbscan_acgt_clustering_min_samples.npy')
    hdbscan_acgt_clustering = np.load(cache_path+'hdbscan_acgt_clustering.npy')

In [ ]:
px.scatter(
    pd.DataFrame({'num_clusters': list(np.max(hdbscan_acgt_clustering, axis = 0)),
                  'samples_in_cluster': list(np.sum(hdbscan_acgt_clustering > -1, axis = 0)),
                  'minumum_samples': [e for e in hdbscan_acgt_clustering_min_samples]}),    
    x = 'num_clusters', y = 'samples_in_cluster', color = 'minumum_samples')

In [ ]:
acgt_clustering_df = pd.concat([
    # distinct hybrids
    phno.loc[:, ['Hybrid', 'Geno_Idx']].drop_duplicates().sort_values('Geno_Idx').reset_index(drop=True),
    # clusterings
    pd.DataFrame(hdbscan_acgt_clustering, columns=['min_'+str(e) for e in list(hdbscan_acgt_clustering_min_samples)])
], axis = 1)

acgt_clustering_df[['Female', 'Male']] = acgt_clustering_df['Hybrid'].str.split('/', expand = True)

acgt_clustering_df.loc[:, ['Hybrid', 'min_16']]

In [ ]:
# What about in this clustering?
temp = acgt_clustering_df.loc[:, ['Female', 'Male', 'min_16']]

In [ ]:
# how many genotypes are in each cluster?
len(set(temp.min_16))

In [ ]:
px.line(pd.DataFrame({'cluster': [str(e) for e in list(set(temp.min_16))],
              'hybrid_count': [temp.loc[(temp.min_16 == i) , 'min_16'].count() for i in list(set(temp.min_16))]
              }).sort_values('hybrid_count', ascending=False),
          x = 'cluster', y = 'hybrid_count', markers='marker'
          )

In [ ]:
temp.loc[temp.min_16 == 107, ]

In [ ]:
parents_in_cluster = pd.concat([temp.loc[:, ['Female', 'min_16']].rename(columns = {'Female':'Parent'}), 
           temp.loc[:, ['Male', 'min_16']].rename(columns = {'Male':'Parent'})]
         ).assign(n=1).drop_duplicates()

px.imshow(
parents_in_cluster.pivot(columns='Parent', index='min_16', values='n').reset_index()
)

In [ ]:
# How mnay parents are exclusively in one cluster?
print('Clusters do not appear to be distinct to prevalent parents (e.g. LH195).\nIf I use clusters by HDBSCAN, then common parents will show up in many clusters:')
parents_in_cluster.groupby(['Parent']).count().sort_values('n', ascending=False).head(20)

## Clustering based on unique parents

In [ ]:
print('Unique Females, Males')
[len(set(acgt_clustering_df[e])) for e in ['Female', 'Male']]

In [ ]:
uniq_parents = list(set(list(set(acgt_clustering_df['Female']))+list(set(acgt_clustering_df['Male']))))
print('Unique Parents: '+str(len(uniq_parents)))

In [ ]:
temp = phno.copy()
temp[['Female', 'Male']] = temp['Hybrid'].str.split('/', expand = True)
temp = temp.loc[:, ['Female', 'Male', 'Yield_Mg_ha']].groupby(['Female', 'Male']).count().reset_index()

temp_count = pd.DataFrame({
    'uniq_parents': uniq_parents,
    'count': [temp.loc[((temp.Female == uniq_parents[i]) | 
                        (temp.Male == uniq_parents[i])), 'Yield_Mg_ha'].sum() 
                        for i in range(len(uniq_parents))]})

print('The vast majority of samples have one of a few parents')
temp_count.sort_values('count', ascending= False).head(20)

In [ ]:
px.line(temp_count.sort_values('count', ascending= False), x = 'uniq_parents', y = 'count', log_y=False)

In [ ]:
px.imshow(temp.pivot(columns='Male', index='Female', values='Yield_Mg_ha').reset_index())

## Train/Test Groups based on Unique parents.

In [ ]:
temp_count.sort_values('count', ascending= False).head(30).T

In [ ]:
temp = phno.loc[:, ['Year', 'Hybrid']].copy()
temp[['Female', 'Male']] = temp['Hybrid'].str.split('/', expand = True)
uniq_parents = list(set(list(temp['Female'])+list(temp['Male'])))
uniq_years = list(set(list(temp['Year'])))
temp = temp.groupby(['Year', 'Female', 'Male']).count().reset_index()
temp

In [ ]:
# take a leaf out of MCMC and set probability to sample a parent to the represenation of that parent in the data set
temp_sums = np.array([temp.loc[((temp.Female == e) | (temp.Male == e)), 'Hybrid'].sum() for e in uniq_parents])
temp_sums = temp_sums/np.sum(temp_sums)

In [ ]:
def tally_cv00(
    draw_years = 1,
    draw_parents = 1,
    seed_val = 234786,
    uniq_years = uniq_years,
    uniq_parents = uniq_parents, 
    tally_df = temp, 
    **kwargs # optional probabilites for np.random.choice for uniq_years, uniq_genos. Should be called p_year, p_parents
):
    np.random.seed(seed_val)
    if 'p_year' in kwargs:
        test_years   = list(np.random.choice(uniq_years,   draw_years, replace = False, p = kwargs['p_year']))
    else:
        test_years   = list(np.random.choice(uniq_years,   draw_years, replace = False))
    
    if 'p_parents' in kwargs:
        test_parents = list(np.random.choice(uniq_parents, draw_parents, replace = False, p = kwargs['p_parents']))
    else:
        test_parents = list(np.random.choice(uniq_parents, draw_parents, replace = False))

    test_year_mask   = tally_df.Year.isin(test_years) 
    test_Female_mask = tally_df.Female.isin(test_parents)
    test_Male_mask   = tally_df.Male.isin(test_parents)

    train_mask = ((~(test_year_mask)) & (~(test_Female_mask | test_Male_mask)))
    test_mask  = (( (test_year_mask)) & ( (test_Female_mask | test_Male_mask)))

    return({
        'n_train': tally_df.loc[train_mask, 'Hybrid'].sum(),
        'n_test': tally_df.loc[test_mask,  'Hybrid'].sum(), 
        'test_years': test_years,
        'test_parents': test_parents
    })

### Test large number of individuals first test set sampling

In [ ]:
# Sampling with non-uniform probabilities
np.random.seed(347899)

n_samples = 1000
res_list = []


for ith_draw_parents in tqdm([max(1, (5*i)) for i in range(5)]):
    for ith_draw_years in [1+i for i in range(2)]:

        res = [tally_cv00(
            draw_years = ith_draw_years,
            draw_parents = ith_draw_parents,
            seed_val = e,
            uniq_years = uniq_years,
            uniq_parents = uniq_parents, 
            tally_df = temp,
        #     p_year
            p_parents = temp_sums
        ) for e in np.random.randint(1, 10000, n_samples)]

        res = pd.DataFrame(zip([e['n_train'] for e in res], [e['n_test'] for e in res]), 
                     columns= ['n_train', 'n_test'])
        res['draw_years'] = ith_draw_years
        res['draw_parents'] = ith_draw_parents
        res_list += [res]
        
        
res = pd.concat(res_list)
# constrain to have the desired test train approximate split
# res = res.loc[((0.09 < (res.n_test / (res.n_train + res.n_test))) & 
#          (0.11 > (res.n_test / (res.n_train + res.n_test)))), ]
# px.scatter(res, x = 'draw_parents', y = 'n_test', facet_col='draw_years')

In [ ]:
res['pr_test'] = res.n_test / (res.n_train + res.n_test)
px.scatter(res, x = 'n_test', y = 'pr_test', color = 'draw_parents', facet_col='draw_years')

### Test rarity first test set sampling

In [ ]:
# Sampling with non-uniform probabilities INVERSE to representation (i.e. highest chance of getting non-testers)
np.random.seed(347899)

n_samples = 1000
res_list = []


for ith_draw_parents in tqdm([int(np.floor(i)) for i in np.linspace(1, 2203, 15)]):
    for ith_draw_years in [1+i for i in range(2)]:

        res = [tally_cv00(
            draw_years = ith_draw_years,
            draw_parents = ith_draw_parents,
            seed_val = e,
            uniq_years = uniq_years,
            uniq_parents = uniq_parents, 
            tally_df = temp,
        #     p_year
            p_parents = ((1/temp_sums)/np.sum((1/temp_sums))) # <-- Now inversely weighted by n_obs (sample testers last)
        ) for e in np.random.randint(1, 10000, n_samples)]

        res = pd.DataFrame(zip([e['n_train'] for e in res], [e['n_test'] for e in res]), 
                     columns= ['n_train', 'n_test'])
        res['draw_years'] = ith_draw_years
        res['draw_parents'] = ith_draw_parents
        res_list += [res]
        
        
res = pd.concat(res_list)
# constrain to have the desired test train approximate split
# res = res.loc[((0.09 < (res.n_test / (res.n_train + res.n_test))) & 
#          (0.11 > (res.n_test / (res.n_train + res.n_test)))), ]

In [ ]:
res['pr_test'] = res.n_test / (res.n_train + res.n_test)
res

In [ ]:
px.scatter(res, x = 'n_test', y = 'pr_test', color = 'draw_parents', facet_col='draw_years')

## Demonstrate rarity based test/train/validate

Initial setup: Need a df with the observation counts for environments and genotypes, and unique parents.

In [ ]:
parent_tally = phno.loc[:, ['Year', 'Hybrid']].copy()
parent_tally[['Female', 'Male']] = parent_tally['Hybrid'].str.split('/', expand = True)

uniq_parents = list(set(list(parent_tally['Female'])+list(parent_tally['Male'])))
uniq_years = list(set(list(parent_tally['Year']))) # this is included for consistency. In practice I will specify the years to be selected from.

parent_tally = parent_tally.groupby(['Year', 'Female', 'Male']).count().reset_index()
parent_tally

### Generate Train/Test set

In [ ]:
# Calculate weights for each of the allowed parents. Weights should be inversely proportionate to the number of childern each parent has.
allowed_parents = uniq_parents 

allowed_parents_weights = np.array([parent_tally.loc[((parent_tally.Female == e) | (parent_tally.Male == e)), 'Hybrid'].sum() for e in allowed_parents])
allowed_parents_weights = allowed_parents_weights/np.sum(allowed_parents_weights) # pr
allowed_parents_weights = (1/allowed_parents_weights)/np.sum(1/allowed_parents_weights) # inv -> pr
allowed_parents_weights[np.isnan(allowed_parents_weights)] = 0
allowed_parents_weights

In [ ]:
# loop until an acceptable test/train split is found. Using an large set of seeds instead of a while loop.

res = None

for i_seed in np.random.randint(1, 10000, 1000):
    if res != None:
        if ((res['n_test'] > 1000) &  # minimum samples and proportion threshold
            (0.09 < (res['n_test'] / (res['n_train'] + res['n_test'])) < 0.11)):
#             print('stop')
            break

    res = tally_cv00(
                draw_years = 2,
                draw_parents = np.random.randint(
                    round(0.1*len(allowed_parents)),
                    round(0.5*len(allowed_parents))),
                seed_val = i_seed,
                uniq_years = [2022, 2021],
                uniq_parents = allowed_parents, 
                tally_df = parent_tally,
                p_parents = allowed_parents_weights
            )

In [ ]:
test_set = res.copy()
print("Set contains:\nTest \tTrain\n"+str(res['n_test'])+"\t"+str(res['n_train']))

### Generate validation sets

In [ ]:
# update allowed parents to exclude those in the test set and repeat several times for possible validation sets.
allowed_parents = [e for e in allowed_parents if e not in res['test_parents']]
# disallow years and parents in res
mask = ((~(parent_tally.Year.isin(res['test_years']))) & 
        (~((parent_tally.Female.isin(res['test_parents']) | parent_tally.Male.isin(res['test_parents'])))))
# overwrite
parent_tally = parent_tally.loc[mask, ]

In [ ]:
parent_tally

In [ ]:
# remove any of the allowed parents that are no longer in the parent tally. This will prevent entries with _no_ children from getting undefined weight 
allowed_parents = [e for e in allowed_parents if e in set(list(parent_tally['Female'])+list(parent_tally['Male']))]

# Repeat to generate a validation set
allowed_parents_weights = np.array([parent_tally.loc[((parent_tally.Female == e) | (parent_tally.Male == e)), 'Hybrid'].sum() for e in allowed_parents])
allowed_parents_weights = allowed_parents_weights/np.sum(allowed_parents_weights) # pr
allowed_parents_weights = (1/allowed_parents_weights)/np.sum(1/allowed_parents_weights) # inv -> pr
allowed_parents_weights[np.isnan(allowed_parents_weights)] = 0

In [ ]:
validation_set_n = 5
validation_sets = []

for validation_set in range(validation_set_n):
    res = None
    for i_seed in np.random.randint(1, 10000, 10000):
        if res != None:
            if ((res['n_test'] > 500) &  # minimum samples and proportion threshold
                (0.09 < (res['n_test'] / (res['n_train'] + res['n_test'])) < 0.11)):
#                 print('stop')
                break

        res = tally_cv00(
                    draw_years = 2,
                    draw_parents = np.random.randint(
                        round(0.1*len(allowed_parents)),
                        round(0.5*len(allowed_parents))), 
                    seed_val = i_seed,
                    uniq_years = [2014, 2015, 2016, 2017, 2018, 2019, 2020],
                    uniq_parents = allowed_parents, 
                    tally_df = parent_tally,
                    p_parents = allowed_parents_weights
                )
    validation_sets += [res]


In [ ]:
print("Set contains:\nTest \tTrain\n----    -----")
_ = [print(str(res['n_test'])+"\t"+str(res['n_train'])) for res in validation_sets]

### Visualize generated results

In [ ]:
# recreate original tally
parent_tally = phno.loc[:, ['Year', 'Hybrid']].copy()
parent_tally[['Female', 'Male']] = parent_tally['Hybrid'].str.split('/', expand = True)
parent_tally = parent_tally.groupby(['Year', 'Female', 'Male']).count().reset_index()
parent_tally.head()

In [ ]:
column_names = ['Test']+['v'+str(i) for i in range(len(validation_sets))]

for i in range(len(column_names)):
    selection_dict = [test_set]+validation_sets
    selection_dict = selection_dict[i]
    column_name = column_names[i]
    
    parent_tally[column_name] = 'Excluded'

    mask = ((parent_tally.Female.isin(selection_dict['test_parents']) | 
             parent_tally.Male.isin(selection_dict['test_parents'])) & 
            (parent_tally.Year.isin(selection_dict['test_years'])))
    parent_tally.loc[mask, column_name] = 'Test'

    mask = (~(parent_tally.Female.isin(selection_dict['test_parents']) | 
             parent_tally.Male.isin(selection_dict['test_parents'])) & 
            ~(parent_tally.Year.isin(selection_dict['test_years'])))
    parent_tally.loc[mask, column_name] = 'Train'
    
    
# overwrite any entries excluded by train/test that are flagged as included in a validation set    
for i in range(len(column_names)):
    column_name = column_names[i]
    if column_name != 'Test':
        parent_tally.loc[(parent_tally.Test == 'Excluded'), column_name] = 'Excluded'
        parent_tally.loc[(parent_tally.Test == 'Test'), column_name] = 'Test'        


In [ ]:
parent_tally

In [ ]:
# test/train level
px.treemap(parent_tally, path=[px.Constant("all"), 'Test', 'Year'], 
                 values='Hybrid', color='Test')

In [ ]:
treemap_list = [px.treemap(parent_tally, path=[px.Constant("all"), 'Test', e, 'Year'], 
                 values='Hybrid', color='Test') for e in ['v'+str(i) for i in range(len(validation_sets))]]
for e in treemap_list:
    e.show()

## Create production Train/test validation sets.

In [ ]:
np.random.seed(3489675896)
for mk_set in tqdm(range(10)):
    
    ### Setup ######################################################################################
    
    parent_tally = phno.loc[:, ['Year', 'Hybrid']].copy()
    parent_tally[['Female', 'Male']] = parent_tally['Hybrid'].str.split('/', expand = True)

    uniq_parents = list(set(list(parent_tally['Female'])+list(parent_tally['Male'])))
    uniq_years = list(set(list(parent_tally['Year']))) # this is included for consistency. In practice I will specify the years to be selected from.

    parent_tally = parent_tally.groupby(['Year', 'Female', 'Male']).count().reset_index()


    ### Generate Train/Test set ####################################################################

    # Calculate weights for each of the allowed parents. Weights should be inversely proportionate to the number of childern each parent has.
    allowed_parents = uniq_parents 

    allowed_parents_weights = np.array([parent_tally.loc[((parent_tally.Female == e) | (parent_tally.Male == e)), 'Hybrid'].sum() for e in allowed_parents])
    allowed_parents_weights = allowed_parents_weights/np.sum(allowed_parents_weights) # pr
    allowed_parents_weights = (1/allowed_parents_weights)/np.sum(1/allowed_parents_weights) # inv -> pr
    allowed_parents_weights[np.isnan(allowed_parents_weights)] = 0


    # loop until an acceptable test/train split is found. Using an large set of seeds instead of a while loop.

    res = None

    for i_seed in np.random.randint(1, 10000, 1000):
        if res != None:
            if ((res['n_test'] > 1000) &  # minimum samples and proportion threshold
                (0.09 < (res['n_test'] / (res['n_train'] + res['n_test'])) < 0.11)):
    #             print('stop')
                break

        res = tally_cv00(
                    draw_years = 2,
                    draw_parents = np.random.randint(
                        round(0.1*len(allowed_parents)),
                        round(0.5*len(allowed_parents))),
                    seed_val = i_seed,
                    uniq_years = [2022, 2021],
                    uniq_parents = allowed_parents, 
                    tally_df = parent_tally,
                    p_parents = allowed_parents_weights
                )

    # convert from numpy ints to base ints for saving as json
    res['n_train'] = int(res['n_train'])
    res['n_test'] = int(res['n_test'])
    res['test_years'] = [int(e) for e in res['test_years']]

    test_set = res.copy()
    # print("Set contains:\nTest \tTrain\n"+str(res['n_test'])+"\t"+str(res['n_train']))

    ### Generate validation sets ###################################################################

    # update allowed parents to exclude those in the test set and repeat several times for possible validation sets.
    allowed_parents = [e for e in allowed_parents if e not in res['test_parents']]
    # disallow years and parents in res
    mask = ((~(parent_tally.Year.isin(res['test_years']))) & 
            (~(parent_tally.Female.isin(res['test_parents']) | parent_tally.Male.isin(res['test_parents']))))
    # overwrite
    parent_tally = parent_tally.loc[mask, ]


    # remove any of the allowed parents that are no longer in the parent tally. This will prevent entries with _no_ children from getting undefined weight 
    allowed_parents = [e for e in allowed_parents if e in set(list(parent_tally['Female'])+list(parent_tally['Male']))]

    # Repeat to generate a validation set
    allowed_parents_weights = np.array([parent_tally.loc[((parent_tally.Female == e) | (parent_tally.Male == e)), 'Hybrid'].sum() for e in allowed_parents])
    allowed_parents_weights = allowed_parents_weights/np.sum(allowed_parents_weights) # pr
    allowed_parents_weights = (1/allowed_parents_weights)/np.sum(1/allowed_parents_weights) # inv -> pr
    allowed_parents_weights[np.isnan(allowed_parents_weights)] = 0

    validation_set_n = 10
    validation_sets = []

    for validation_set in range(validation_set_n):
        res = None
        for i_seed in np.random.randint(1, 10000, 10000):
            if res != None:
                if ((res['n_test'] > 500) &  # minimum samples and proportion threshold
                    (0.09 < (res['n_test'] / (res['n_train'] + res['n_test'])) < 0.11)):
    #                 print('stop')
                    break

            res = tally_cv00(
                        draw_years = 2,
                        draw_parents = np.random.randint(
                            round(0.1*len(allowed_parents)),
                            round(0.5*len(allowed_parents))), 
                        seed_val = i_seed,
                        uniq_years = [2014, 2015, 2016, 2017, 2018, 2019, 2020],
                        uniq_parents = allowed_parents, 
                        tally_df = parent_tally,
                        p_parents = allowed_parents_weights
                    )
            # convert from numpy ints to base ints for saving as json
        res['n_train'] = int(res['n_train'])
        res['n_test'] = int(res['n_test'])
        res['test_years'] = [int(e) for e in res['test_years']]
        validation_sets += [res]

    ### Save out ###################################################################################

    # use ctime as a unique id for these
    xx = time.localtime()
    xx = ':'.join([str(getattr(xx , e)) for e in ['tm_year', 'tm_mon', 'tm_mday', 'tm_hour','tm_min','tm_sec']])

    with open(cache_path+xx+'-'+'test'+'.json', 'w') as fp:
        json.dump(test_set, fp)

    for i in range(len(validation_sets)):
        with open(cache_path+xx+'-'+'val'+str(i)+'.json', 'w') as fp:
            json.dump(validation_sets[i], fp)

In [ ]:
# find all test/val jsons
jsons = [e for e in os.listdir(cache_path) if re.match('\d+:\d+:\d+:\d+:\d+:\d+-.+\.json$', e)]
temp_prefixes = [e.replace('-test.json', '') for e in jsons if re.match('.+test\.json$', e)]
temp_prefixes

In [ ]:
def load_json(json_path):
    with open(json_path, 'r') as fp:
        dat = json.load(fp)
    return(dat)

In [ ]:
temp_list = []

temp = os.listdir(cache_path)

temp_prefix = '2023:9:5:12:8:26'
for temp_prefix in temp_prefixes:
    print(temp_prefix)
    
    temp_vals = [e for e in temp if re.match('^'+temp_prefix+'-val\d+.json$', e)]
    temp_vals.sort()


    val_n = pd.DataFrame(
        [[load_json(json_path= cache_path+temp_val)[e] for e in ['n_train', 'n_test']] for temp_val in temp_vals], 
        columns=['ValTrain', 'ValTest'])

    test_n = pd.DataFrame(
        [[load_json(json_path= cache_path+temp_prefix+'-test.json')[e] for e in ['n_train', 'n_test']]], 
        columns=['Train', 'Test'])

    val_n['Train'] = int(test_n['Train']) 
    val_n['Test'] = int(test_n['Test']) 
    val_n['Prefix'] = temp_prefix
    
    temp_list += [val_n]
     

In [ ]:
temp =  pd.concat(temp_list).reset_index(drop = True)
temp

In [ ]:
temp = temp.melt(id_vars=['Prefix'], value_vars=['ValTrain', 'ValTest', 'Train', 'Test']).drop_duplicates()

In [ ]:
temp

In [ ]:
px.scatter(temp, x = 'Prefix', y = 'value', color = 'variable' )